In [3]:
import requests
import json
import time

In [4]:
def get_all_business_ids(base_url):
    
    all_business_ids = []

    while True:
        response = requests.get(base_url, headers=headers)
        if response.status_code != 200:
            break  

        data = response.json()
        businesses = data.get('businesses', [])
        if not businesses:
            break  # Break the loop if no more businesses are returned

        for business in businesses:
            business_id = business.get('id')
            if business_id:
                all_business_ids.append(business_id)

        # Update the offset in the URL for the next request
        if 'offset=' in base_url:
            base_url = base_url.rsplit('offset=', 1)[0] + f'offset={len(all_business_ids)}'
        else:
            base_url += f'&offset={len(all_business_ids)}'

        time.sleep(1)  

    return all_business_ids


['ETgJqJHV7BW6pIr9Ox74sA', 'DGhWO1sUWydVeR5j5ZZaMw', 'KgpOYAG-r_eDsQXFXt0nnQ', 'ZjU9qd-RR55cEEKufmdN5Q', 'FiRB9Ek3Xo9JHwXgBYITYg', 's3jou_L_LVYGkNHiuhjlew', 'npiIhpcWePbVT3JYEcy3zg', '8XHbZIKDQbXhyKIGiukVeQ', 'PflHKSowWSsR-a76Ne7nzg', 'EF4M55USa1ZNy1rP5Kuk2w', 'V3dHB6xAdd8_dH7t481PAw', 'wDLHOgux_G8Y82Y6D9rSog', 'gtXP0CmOnBbgfnff7VXa7A', 'TQzGf4k3HXyMbnNkYKKQSw', 'PYcL2z9gHj6-ZTlViT9Nqw', 'aTFkjo0scqmOgsFF5kuGXg', 'P2CTp4xyaNCe2AXSiLeOSQ', 'cHvd_fnNqbcR4EdyodzGzw', 'obFs12x7mgYPGxgalBd22w', 'nBkSqOhZMa1A7Yk3q8VPaA', 'mxDfu4bHwSXoCr3FRpyMsw', 'CcxreLJbZsO6Z775-hMWpw', 'C6f_MQaRwIbJ8diailVaCA', 'g5f6p9jfzB30Mhcu6d5OWA', 'GBJOu9Rk4JqqWjB8ddYudw', 'HJg9TbpyWvb6-AFPdb7NCA', 'q-QEwZi32JCnjb7zbY-PQw', 'mjGdtrh0tX7I91GM9dlMNg', 'tmzWZtu9Nimn7Hfas9XmcA', 'OMv26_R8kaek1G7_QhBPkg', '7N6Jle8ovxzAiiwwtE77Yw', 'qLL_x4uzDHCgXd9lwaGxjw', 'kMljfja7weqfl_lzspjzHA', '50fWXrNECQQYght4ZWeBXQ', 'NMVRHrkdp68twl4CWo1I5A', '2VpurA2lb-q90nmALklNcA', 'l2b6HJGarUxoeLnM9syTmg', 'BA6wd1pZTVu1DmrtiyTs8g', 'FON3TceJNs

In [ ]:
api_key = 'rjj-CYij9D5rBs0P7jLtSGO6u2hBp38A5jMZTGELqjvvojBqfepQac4jprE6LhD84IuHE9UuswLpOL46Ou8IA04Jd4Ije97e5rv5Q4Ifag8tQrMlJirC2-eTUH58ZXYx'
base_url = (
    'https://api.yelp.com/v3/businesses/search?'
    'location=New+York+City&term=restaurants&categories=french'
    '&price=3&price=4&sort_by=best_match&limit=50'
)
headers = {'Authorization': 'Bearer ' + api_key}
nyc_restaurant_ids = get_all_business_ids(base_url)
print(nyc_restaurant_ids)

In [36]:
len(nyc_restaurant_ids)

97

In [11]:
def get_reviews(restaurant_ids):
    list_of_reviews = []
    for i in range(len(restaurant_ids)):
        url2 = "https://api.yelp.com/v3/businesses/" + restaurant_ids[i] + "/reviews?limit=50"
        print(url2)
        response = requests.get(url2, headers=headers)
        reviews_data = response.json()
        try:
            reviews_text = [review['text'] for review in reviews_data['reviews']]
            for review_text in reviews_text:
                list_of_reviews.append(review_text)
        except:
            print("No reviews for this restaurant")
        
    return list_of_reviews


In [ ]:
nyc_list_of_reviews = get_reviews(nyc_restaurant_ids)

### San Francisco

In [12]:
sf_url = "https://api.yelp.com/v3/businesses/search?location=San+Francisco&term=restaurants&categories=french&price=4&price=3&sort_by=best_match&limit=50"

san_francisco_restaurants = get_all_business_ids(sf_url)

sf_list_of_reviews = get_reviews(san_francisco_restaurants)

https://api.yelp.com/v3/businesses/cCVR9W_L2rqexfKAjfBTzQ/reviews?limit=50
https://api.yelp.com/v3/businesses/9yM20-7fj4LMrOO30nqkBw/reviews?limit=50
https://api.yelp.com/v3/businesses/WavvLdfdP6g8aZTtbBQHTw/reviews?limit=50
https://api.yelp.com/v3/businesses/L5vSeC_sa3TUQW_lticqyA/reviews?limit=50
https://api.yelp.com/v3/businesses/muX0aRES6vsp5DEuJaW_uw/reviews?limit=50
https://api.yelp.com/v3/businesses/9e2UrZrwbWsTyN3hWyS6Ig/reviews?limit=50
https://api.yelp.com/v3/businesses/c3khi7kxBaA2ATUuMjtMZA/reviews?limit=50
https://api.yelp.com/v3/businesses/oqsu3pKpgRMGHj9QItsx0A/reviews?limit=50
https://api.yelp.com/v3/businesses/353bun0151fMbR3ut2d-nw/reviews?limit=50
https://api.yelp.com/v3/businesses/SG2-29u7Uw1VR9e1xCAT5w/reviews?limit=50
https://api.yelp.com/v3/businesses/tpuKzQh5d6veDB-M9qU6Fw/reviews?limit=50
https://api.yelp.com/v3/businesses/mfD5wlWS6tY92FJHDcBwJA/reviews?limit=50
https://api.yelp.com/v3/businesses/xO2V0wkXP3Na-mWhLo1ZxQ/reviews?limit=50
https://api.yelp.com/v3/b

In [13]:
print(len(sf_list_of_reviews))

75


### Miami

In [6]:
miami_url = "https://api.yelp.com/v3/businesses/search?location=Miami&term=restaurants&categories=french&price=4&price=3&sort_by=best_match&limit=50"

miami_restaurants = get_all_business_ids(miami_url)

miami_list_of_reviews = get_reviews(miami_restaurants)

https://api.yelp.com/v3/businesses/-QtB-fGXYKdaEQySVO28wg/reviews?limit=50
https://api.yelp.com/v3/businesses/uyaz8BMIp2n7lhSpjZ8quA/reviews?limit=50
https://api.yelp.com/v3/businesses/GRZpClfFf3otLO2ridn5iA/reviews?limit=50
https://api.yelp.com/v3/businesses/B2-pidnIEJcI4-sy97BJww/reviews?limit=50
https://api.yelp.com/v3/businesses/K6RzqmTJ5yaI35Bw1BAE3g/reviews?limit=50


In [7]:
print(len(miami_list_of_reviews))

15


### Los Angeles

In [8]:
los_angeles_url = "https://api.yelp.com/v3/businesses/search?location=Los+Angeles&term=restaurants&categories=french&price=4&price=3&sort_by=best_match&limit=50"

los_angeles_restaurants = get_all_business_ids(los_angeles_url)

los_angeles_list_of_reviews = get_reviews(los_angeles_restaurants)

https://api.yelp.com/v3/businesses/K-Fbh2WCHlqR56H9TnfpNA/reviews?limit=50
https://api.yelp.com/v3/businesses/LJEsTC6Juzr0lL-hla7nVw/reviews?limit=50
https://api.yelp.com/v3/businesses/-ZcCFCdqMwOPXItB3REecA/reviews?limit=50
https://api.yelp.com/v3/businesses/gCFw0Tx6h4E8oNwDYkIRTg/reviews?limit=50
https://api.yelp.com/v3/businesses/XrPVlXXa4cHLm7qKWFkfCg/reviews?limit=50
https://api.yelp.com/v3/businesses/fDYbMJqg7WZ4wek2QN3t1w/reviews?limit=50
https://api.yelp.com/v3/businesses/SREXeeGWWQwKP54fCljwZg/reviews?limit=50
https://api.yelp.com/v3/businesses/OVYbBxiA-MFNE3wSKlSlgg/reviews?limit=50
https://api.yelp.com/v3/businesses/6RptZH8sx1vnQ_io_HrmeA/reviews?limit=50
https://api.yelp.com/v3/businesses/lq1MXuvw45E9SfpQL-LMgw/reviews?limit=50
https://api.yelp.com/v3/businesses/kBJdl7y-MALnpKwU_HcYkA/reviews?limit=50
https://api.yelp.com/v3/businesses/cjOPYO30o3HBDhVvsy7Hbw/reviews?limit=50
https://api.yelp.com/v3/businesses/LY7ERi0y0OPzk-pNhFadDw/reviews?limit=50
https://api.yelp.com/v3/b

In [9]:
print(len(los_angeles_list_of_reviews))

93


In [27]:
seattle_url = "https://api.yelp.com/v3/businesses/search?location=Washington&term=restaurants&categories=french&price=4&price=3&sort_by=best_match&limit=50"

seattle_restaurants = get_all_business_ids(seattle_url)

In [28]:
print(len(seattle_restaurants))

17
